In [1]:
# for validation
import numpy as np
import pandas as pd
import datetime
from netCDF4 import Dataset,num2date,date2num
import gpflow as gf
from eofs.standard import Eof
import time
from datetime import datetime, timedelta
import numpy.ma as ma
import os
from typing import Sequence
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [7]:
# Load historical simulation data
historical_simulation_data = Dataset('D:\CWatM-new\Brahmaputra\output228\discharge_daily_ipsl.nc','r')

# Load historical observation data
historical_observation_data = Dataset('E:\data\era5_runoff\disdata\era5_data1979-2014_5min.nc','r')

ipsl_hist_train_origin = historical_simulation_data.variables['discharge'][:]
lons = historical_simulation_data.variables['lon'][:]
lats = historical_simulation_data.variables['lat'][:]

obsdata_origin = historical_observation_data.variables['dis24'][:]
historical_observation_data.close()
historical_simulation_data.close()

mask_t = ipsl_hist_train_origin == ipsl_hist_train_origin[1][0][0]
ipsl_hist_train = ma.array(ipsl_hist_train_origin,mask = mask_t)
obsdata = ma.array(obsdata_origin,mask = mask_t)
#数据切片调试-可以不做
#ipsl_hist_train_old = ma.array(ipsl_hist_train_origin,mask = mask_t)
#obsdata_old = ma.array(obsdata_origin,mask = mask_t)
#ipsl_hist_train = ipsl_hist_train_old[13000:][:][:]
#obsdata = obsdata_old[13000:][:][:]

In [8]:
#提取两个数据的主成分，减少分析两个数据差异的复杂度
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
#coslat = np.cos(np.deg2rad(lats)).clip(0., 1.)
#wgts = np.sqrt(coslat)[..., np.newaxis]


# Perform EOF analysis on data1 and data2
#obsdata_solver = Eof(obsdata, weights=None, center=False)
obsdata_solver = Eof(obsdata, weights=None,center=False)

#explain for how much variance
obsdata_mode = obsdata_solver.varianceFraction(neigs=4)
print(obsdata_mode)

# Retrieve the leading EOFs for both datasets
obsdata_eofs = obsdata_solver.eofs(neofs=4)
print(obsdata_eofs.shape)

#variance correlation
obsdata_eofc = obsdata_solver.eofsAsCorrelation(neofs=4)


#create pseudo-ECs for GPR

#源代码中obsdata 不存在 obsdata -> obsdata_old
ipsl_hist_train_ECs = obsdata_solver.projectField(ipsl_hist_train,neofs = 4)
obsdata_ECs = obsdata_solver.projectField(obsdata,neofs = 4)
print(ipsl_hist_train_ECs.shape)

[0.9868846  0.0065986  0.00354106 0.00170928]
(4, 89, 189)
(13149, 4)


In [4]:
#将两列ECs各自做为训练集
ipsl_hist_train_ECs_l1 = ipsl_hist_train_ECs[:,0].reshape(13149,1)
ipsl_hist_train_ECs_x1 = tf.convert_to_tensor(ipsl_hist_train_ECs_l1)
ipsl_hist_train_ECs_x1=tf.cast(ipsl_hist_train_ECs_x1,tf.float64)
ipsl_hist_train_ECs_l2 = ipsl_hist_train_ECs[:,1].reshape(13149,1)
ipsl_hist_train_ECs_x2 = tf.convert_to_tensor(ipsl_hist_train_ECs_l2)
ipsl_hist_train_ECs_x2=tf.cast(ipsl_hist_train_ECs_x2,tf.float64)
ipsl_hist_train_ECs_l3 = ipsl_hist_train_ECs[:,2].reshape(13149,1)
ipsl_hist_train_ECs_x3 = tf.convert_to_tensor(ipsl_hist_train_ECs_l3)
ipsl_hist_train_ECs_x3=tf.cast(ipsl_hist_train_ECs_x3,tf.float64)
ipsl_hist_train_ECs_l4 = ipsl_hist_train_ECs[:,3].reshape(13149,1)
ipsl_hist_train_ECs_x4 = tf.convert_to_tensor(ipsl_hist_train_ECs_l4)
ipsl_hist_train_ECs_x4=tf.cast(ipsl_hist_train_ECs_x4,tf.float64)
obsdata_ECs_l1 = obsdata_ECs[:,0].reshape(13149,1)
obsdata_ECs_y1 = tf.convert_to_tensor(obsdata_ECs_l1)
obsdata_ECs_y1=tf.cast(obsdata_ECs_y1,tf.float64)
obsdata_ECs_l2 = obsdata_ECs[:,1].reshape(13149,1)
obsdata_ECs_y2 = tf.convert_to_tensor(obsdata_ECs_l2)
obsdata_ECs_y2=tf.cast(obsdata_ECs_y2,tf.float64)
obsdata_ECs_l3 = obsdata_ECs[:,2].reshape(13149,1)
obsdata_ECs_y3 = tf.convert_to_tensor(obsdata_ECs_l3)
obsdata_ECs_y3=tf.cast(obsdata_ECs_y3,tf.float64)
obsdata_ECs_l4 = obsdata_ECs[:,3].reshape(13149,1)
obsdata_ECs_y4 = tf.convert_to_tensor(obsdata_ECs_l4)
obsdata_ECs_y4=tf.cast(obsdata_ECs_y4,tf.float64)

In [5]:
#EOF1
#standardize x,y
n_input_ECs = 4
scaler_x = StandardScaler()
scaler_y = StandardScaler()
ipsl_train = np.c_[ipsl_hist_train_ECs_x1,ipsl_hist_train_ECs_x2,ipsl_hist_train_ECs_x3,ipsl_hist_train_ECs_x4]
ipsl_train_sc = scaler_x.fit_transform(ipsl_train)
#inducing（修改）
n_inducing_points=700
model_ipsl_eof = list()
obs_train_all = np.c_[obsdata_ECs_y1,obsdata_ECs_y2,obsdata_ECs_y3,obsdata_ECs_y4]
for i in range(n_input_ECs):
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(ipsl_train_sc[:,0].min(), ipsl_train_sc[:,0].max(), n_inducing_points)]
    for j in range(1, n_input_ECs):
            inducing_points = np.c_[inducing_points,
                                    np.linspace(ipsl_train_sc[:, j].min(), ipsl_train_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(ipsl_train_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (ipsl_train_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_ipsl_eof.append(model)
    start_time = time.monotonic()
    opt = gf.optimizers.Scipy()
    opt.minimize(model_ipsl_eof[i].training_loss, model_ipsl_eof[i].trainable_variables)
    gf.utilities.print_summary(model_ipsl_eof[i], "notebook")
    opt_time_eof1 = time.monotonic()
    opt_ipsl_eof1_timeperiod = opt_time_eof1 - start_time
    print("The ",i+1," ECs training for obsdata")
    print('opt_ipsl_eof1_timeperiod:',opt_ipsl_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_ipsl_eof[i].predict_y(ipsl_train_sc, full_cov=False)
    ipsl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(ipsl_pred_mean) / np.mean(obs_train)
    y = (np.std(ipsl_pred_mean) / np.mean(ipsl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.87544
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.82635
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.23858
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.13218, -5.89456, -11.44182..."


The  1  ECs training for obsdata
opt_ipsl_eof1_timeperiod: 57318.937000000005
0.9984585834284709 0.8671089517900953


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.89401
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.71915
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.6402
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.04068, -6.10387, -11.33384..."


The  2  ECs training for obsdata
opt_ipsl_eof1_timeperiod: 35964.03100000002
1.0654320844589797 0.5383189663000822


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.46841
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.56667
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.78989
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.12127, -5.96236, -11.32813..."


The  3  ECs training for obsdata
opt_ipsl_eof1_timeperiod: 46558.625
0.9980151881123382 0.4086602376009834


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.53951
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.6084
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.768
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.11801, -5.91728, -11.4055..."


The  4  ECs training for obsdata
opt_ipsl_eof1_timeperiod: 37171.79599999997
1.021839927749666 0.4297771360912205


In [6]:
ipsl_pred_ECs = np.zeros((13149,n_input_ECs))
ipsl_pred_ECs_lb = np.zeros((13149,n_input_ECs))
ipsl_pred_ECs_ub = np.zeros((13149,n_input_ECs))
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

for i in range(n_input_ECs):
    y_pred_mean, y_pred_var= model_ipsl_eof[i].predict_y(ipsl_train_sc, full_cov=False)
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    print(y_pred_mean,obs_train_sc)
    ipsl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    print(ipsl_pred_mean,obs_train)
    B = np.mean(ipsl_pred_mean) / np.mean(obs_train)
    y = (np.std(ipsl_pred_mean) / np.mean(ipsl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)
    ipsl_pred_ECs[:,i] =np.squeeze(ipsl_pred_mean)
    print(ipsl_pred_ECs[:,i])
    y__pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    ipsl_pred_ECs_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    ipsl_pred_ECs_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
#合并校正后的两个主成分-
ipsl_hist_train_ECs_SGPRcor=tf.concat([ipsl_pred_ECs[:,0].reshape(13149, 1),ipsl_pred_ECs[:,1].reshape(13149, 1),ipsl_pred_ECs[:,2].reshape(13149, 1),ipsl_pred_ECs[:,3].reshape(13149, 1)],axis=1)
ipsl_hist_train_ECs_lb_SGPRcor=tf.concat([ipsl_pred_ECs_lb[:,0].reshape(13149, 1),ipsl_pred_ECs_lb[:,1].reshape(13149, 1),ipsl_pred_ECs_lb[:,2].reshape(13149, 1),ipsl_pred_ECs_lb[:,3].reshape(13149, 1)],axis=1)
ipsl_hist_train_ECs_ub_SGPRcor=tf.concat([ipsl_pred_ECs_ub[:,0].reshape(13149, 1),ipsl_pred_ECs_ub[:,1].reshape(13149, 1),ipsl_pred_ECs_ub[:,2].reshape(13149, 1),ipsl_pred_ECs_ub[:,3].reshape(13149, 1)],axis=1)

df_mean_merge = pd.DataFrame(ipsl_hist_train_ECs_SGPRcor,columns = ['ipsl_EC1','ipsl_EC2','ipsl_EC3','ipsl_EC4'])
df_lb_merge = pd.DataFrame(ipsl_hist_train_ECs_lb_SGPRcor,columns = ['ipsl_EC1_lb','ipsl_EC2_lb','ipsl_EC3_lb','ipsl_EC4_lb'])
df_ub_merge = pd.DataFrame(ipsl_hist_train_ECs_ub_SGPRcor,columns = ['ipsl_EC1_ub','ipsl_EC2_ub','ipsl_EC3_ub','ipsl_EC4_ub'])
df_obsmean_merge = pd.DataFrame(obs_train_all,columns = ['obs_EC1','obs_EC2','obs_EC3','obs_EC4'])
# 写入Excel文件
df_mean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_mean.xlsx',index=False)
df_lb_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_lb.xlsx',index=False)
df_ub_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_ub.xlsx',index=False)
df_obsmean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\obs_ECs.xlsx',index=False)

tf.Tensor(
[[0.82426083]
 [0.83575985]
 [0.8483679 ]
 ...
 [0.87259812]
 [0.87466046]
 [0.87661688]], shape=(13149, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[[-42117.41507261]
 [-40516.73231488]
 [-38761.67197911]
 ...
 [-35388.78693854]
 [-35101.70617935]
 [-34829.36887406]] [[-26780.41015625]
 [-26305.3046875 ]
 [-25864.76757812]
 ...
 [-31726.390625  ]
 [-31462.87109375]
 [-31136.0703125 ]]
0.9984585834284709 0.8671089517900953
[-42117.41507261 -40516.73231488 -38761.67197911 ... -35388.78693854
 -35101.70617935 -34829.36887406]
tf.Tensor(
[[0.17064013]
 [0.15318036]
 [0.13318973]
 ...
 [0.06771321]
 [0.06733802]
 [0.06712583]], shape=(13149, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[[2552.20190862]
 [2252.86506327]
 [1910.13848409]
 ...
 [ 787.58490531]
 [ 781.15265612]
 [ 777.51474337]] [[-2039.22070312]
 [-1982.69091797]
 [-1931.41503906]
 

In [8]:
#第二次GPFLOW：obs-mean/lb/ub
scaler_z = StandardScaler()
ipsl_pred_ECs_2GP = np.zeros((13149,n_input_ECs))
ipsl_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
ipsl_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
n_inducing_points=700
model_ipsl_eof_2GP = list()
ipsl_pred_ECs_range = np.zeros((13149,n_input_ECs))
for i in range(n_input_ECs):
    ipsl_pred_ECs_range[:,i] = np.subtract(ipsl_pred_ECs_ub[:,i],ipsl_pred_ECs_lb[:,i])
    ipsl_train_2GP = np.c_[ipsl_pred_ECs[:,i],ipsl_pred_ECs_lb[:,i],ipsl_pred_ECs_ub[:,i],ipsl_pred_ECs_range[:,i] ]
    ipsl_train_2GP_sc = scaler_z.fit_transform(ipsl_train_2GP)
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(ipsl_train_2GP_sc[:,0].min(), ipsl_train_2GP_sc[:,0].max(), n_inducing_points)]
    for j in range(3):
        inducing_points = np.c_[inducing_points,
                                    np.linspace(ipsl_train_2GP_sc[:, j].min(), ipsl_train_2GP_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(ipsl_train_2GP_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (ipsl_train_2GP_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_ipsl_eof_2GP.append(model)
    start_time_2GP = time.monotonic()
    opt_2GP = gf.optimizers.Scipy()
    opt_2GP.minimize(model_ipsl_eof_2GP[i].training_loss, model_ipsl_eof_2GP[i].trainable_variables)
    gf.utilities.print_summary(model_ipsl_eof_2GP[i], "notebook")
    opt_time_eof1_2GP = time.monotonic()
    opt_ipsl_eof1_2GP_timeperiod = opt_time_eof1_2GP - start_time_2GP
    print("The ",i+1," ECs training for obsdata")
    print('opt_ipsl_eof1_2GP_timeperiod:',opt_ipsl_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_ipsl_eof_2GP[i].predict_y(ipsl_train_2GP_sc, full_cov=False)
    ipsl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(ipsl_pred_mean) / np.mean(obs_train)
    y = (np.std(ipsl_pred_mean) / np.mean(ipsl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    ipsl_pred_ECs_2GP[:,i] =np.squeeze(ipsl_pred_mean)
    print(ipsl_pred_ECs_2GP[:,i])
    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    ipsl_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    ipsl_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
    print(B,y)
    

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,149.75586
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,38.52893
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.22581
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.78646e+00, 2.50210e-01, -5.78615e+00..."


The  1  ECs training for obsdata
opt_ipsl_eof1_2GP_timeperiod: 37171.79599999997
[-42615.18400869 -41117.58254434 -39480.12984873 ... -36352.02173088
 -36085.12478119 -35831.92246236]
0.9999983176379758 0.8798272735515578


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,6.50798
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,13.68441
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.60503
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-4.1948, -4.23333, -4.25395..."


The  2  ECs training for obsdata
opt_ipsl_eof1_2GP_timeperiod: 37171.79599999997
[2727.90782952 2408.34024283 2041.60305808 ...  837.9893406   830.99333912
  827.00129469]
0.9999085132214894 0.6282685919714617


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,7.90512
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,7.5256
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.73084
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-7.33506, -7.30044, -7.2672..."


The  3  ECs training for obsdata
opt_ipsl_eof1_2GP_timeperiod: 37171.79599999997
[2818.1735708  2878.52887353 2922.16951361 ... 2782.67624712 2780.90086029
 2779.9192275 ]
1.0001220690912909 0.5181476942033912


name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,32.92488
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,29.22518
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.72035
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.59146, -4.83444, -6.9919..."


The  4  ECs training for obsdata
opt_ipsl_eof1_2GP_timeperiod: 37171.79599999997
[1705.18208658 1440.70258084 1149.84358466 ...  191.71709661  170.02034095
  151.76121117]
1.0000507399438903 0.5285122034072952


In [9]:
#ipsl_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
#ipsl_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

#for i in range(n_input_ECs):
#    y_pred_mean, y_pred_var= model_ipsl_eof_2GP[i].predict_y(ipsl_train_2GP_sc, full_cov=False)
#    obs_train = obs_train_all[:,i].reshape(13149,1)
#    obs_train_sc = scaler_y.fit_transform(obs_train)
#    print(y_pred_mean,obs_train_sc)
#    ipsl_pred_mean_2GP = scaler_y.inverse_transform(y_pred_mean)
#    print(ipsl_pred_mean_2GP,obs_train)
#    B = np.mean(ipsl_pred_mean_2GP) / np.mean(obs_train)
#    y = (np.std(ipsl_pred_mean_2GP) / np.mean(ipsl_pred_mean_2GP)) / (np.std(obs_train) / np.mean(obs_train))
#    print(B,y)
#    ipsl_pred_ECs_2GP[:,i] =np.squeeze(ipsl_pred_mean_2GP)
#    print(ipsl_pred_ECs_2GP[:,i])
#    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
#    y_pred_lb = y_pred_mean - y__pred_conf
#    y_pred_ub = y_pred_mean + y__pred_conf
#    ipsl_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
#    ipsl_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
#合并校正后的两个主成分-
ipsl_hist_train_ECs_2GP_SGPRcor=tf.concat([ipsl_pred_ECs_2GP[:,0].reshape(13149, 1),ipsl_pred_ECs_2GP[:,1].reshape(13149, 1),ipsl_pred_ECs_2GP[:,2].reshape(13149, 1),ipsl_pred_ECs_2GP[:,3].reshape(13149, 1)],axis=1)
ipsl_hist_train_ECs_2GP_lb_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_lb[:,0].reshape(13149, 1),ipsl_pred_ECs_2GP_lb[:,1].reshape(13149, 1),ipsl_pred_ECs_2GP_lb[:,2].reshape(13149, 1),ipsl_pred_ECs_2GP_lb[:,3].reshape(13149, 1)],axis=1)
ipsl_hist_train_ECs_2GP_ub_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_ub[:,0].reshape(13149, 1),ipsl_pred_ECs_2GP_ub[:,1].reshape(13149, 1),ipsl_pred_ECs_2GP_ub[:,2].reshape(13149, 1),ipsl_pred_ECs_2GP_ub[:,3].reshape(13149, 1)],axis=1)

df_mean_merge = pd.DataFrame(ipsl_hist_train_ECs_2GP_SGPRcor,columns = ['ipsl_EC1_2GP','ipsl_EC2_2GP','ipsl_EC3_2GP','ipsl_EC4_2GP'])
df_lb_merge = pd.DataFrame(ipsl_hist_train_ECs_2GP_lb_SGPRcor,columns = ['ipsl_EC1_2GP_lb','ipsl_EC2_2GP_lb','ipsl_EC3_2GP_lb','ipsl_EC4_2GP_lb'])
df_ub_merge = pd.DataFrame(ipsl_hist_train_ECs_2GP_ub_SGPRcor,columns = ['ipsl_EC1_2GP_ub','ipsl_EC2_2GP_ub','ipsl_EC3_2GP_ub','ipsl_EC4_2GP_ub'])
df_obsmean_merge = pd.DataFrame(obs_train_all,columns = ['obs_EC1','obs_EC2','obs_EC3','obs_EC4'])
# 写入Excel文件
df_mean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_mean_2GP.xlsx',index=False)
df_lb_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_lb_2GP.xlsx',index=False)
df_ub_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_ub_2GP.xlsx',index=False)
df_obsmean_merge.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\obs_ECs.xlsx',index=False)

In [10]:
#predict
#生成新数据
def ReconstructDataFromEOFandECs(EOFs, ECs):
    # Check if EOF and EC has equally many modes
    if EOFs.shape[0] != ECs.shape[1]:
        return print('EOF and ECs does not have equal number of modes')
    # Project ECs on EOFs
    #recon = np.dot(ECs, EOFs)
    recon = np.zeros((13149,89,189))
    for i in range (len(ECs)):
        recon[i,:,:] = ECs[i,0]*EOFs[0,:,:] + ECs[i,1]*EOFs[1,:,:] + ECs[i,2]*EOFs[2,:,:] + ECs[i,3]*EOFs[3,:,:]
    # Remove weight
    #recon = recon / Weights
    #recon = np.add(recon, Mean)
    # Return reconstructed values
    return recon


In [11]:
recondata_SGPR_ipsl_mix4_2GP = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_hist_train_ECs_2GP_SGPRcor)

In [12]:
#将新数据写入文件
def writeNetCDF(myvar,filename):
    lon = np.linspace(82,97.75,189)
    #lat = np.linspace(23.833333333333,31.25,89)
    lat = np.linspace(31.25,23.833333333333,89)
    start_date = datetime(1979, 1, 1)
    end_date = datetime(2014, 12, 31)
    delta = timedelta(days=1)
    dates = np.arange(start_date, end_date + delta, delta).astype(datetime)
    ncout = Dataset(filename,'w','NETCDF4') # using netCDF4 for output format 
    ncout.createDimension('lon',len(lon))
    ncout.createDimension('lat',len(lat))
    ncout.createDimension('time',None)
    lonvar = ncout.createVariable('lon','float32',('lon'))
    lonvar[:] = lon
    latvar = ncout.createVariable('lat','float32',('lat'))
    latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time',))
    timevar.units = 'days since 1979-01-01 00:00:00'
    timevar.calendar = 'standard'
    timevar[:] = date2num(dates, units=timevar.units, calendar=timevar.calendar)
    var = ncout.createVariable('discharge','float32',('time','lat','lon'))
    var.setncattr('units','m3 s-1')
    var[:] = myvar[:]
    ncout.close()

In [13]:
out_dir='E:/data/era5_runoff/disdata/' 
out_file_nc = out_dir+'recondata_SGPR_ipsl_mix4_2GP.nc' 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP,out_file_nc) 

# 做未来预测

In [14]:
#读入数据
ssp126_ipsl_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_ipsl_ssp126.nc','r')
ssp370_ipsl_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_ipsl_ssp370.nc','r')
ssp585_ipsl_simulation_data = Dataset('E:\data\era5_runoff\discharge_daily_ipsl_ssp585.nc','r')

ipsl_ssp126_train_origin = ssp126_ipsl_simulation_data.variables['discharge'][:]
ipsl_ssp370_train_origin = ssp370_ipsl_simulation_data.variables['discharge'][:]
ipsl_ssp585_train_origin = ssp585_ipsl_simulation_data.variables['discharge'][:]
lons = historical_simulation_data.variables['lon'][:]
lats = historical_simulation_data.variables['lat'][:]

mask_t = ipsl_ssp126_train_origin == ipsl_ssp126_train_origin[1][0][0]

ipsl_ssp126_train = ma.array(ipsl_ssp126_train_origin,mask = mask_t)
ipsl_ssp370_train = ma.array(ipsl_ssp370_train_origin,mask = mask_t)
ipsl_ssp585_train = ma.array(ipsl_ssp585_train_origin,mask = mask_t)

ipsl_ssp126_train_ECs = obsdata_solver.projectField(ipsl_ssp126_train,neofs = 4)
ipsl_ssp370_train_ECs = obsdata_solver.projectField(ipsl_ssp370_train,neofs = 4)
ipsl_ssp585_train_ECs = obsdata_solver.projectField(ipsl_ssp585_train,neofs = 4)

ssp126_ipsl_simulation_data.close()
ssp370_ipsl_simulation_data.close()
ssp585_ipsl_simulation_data.close()

In [15]:
#将两列ECs各自做为训练集
ipsl_ssp126_train_ECs_l1 = ipsl_ssp126_train_ECs[:,0].reshape(28854,1)
ipsl_ssp126_train_ECs_x1 = tf.convert_to_tensor(ipsl_ssp126_train_ECs_l1)
ipsl_ssp126_train_ECs_x1=tf.cast(ipsl_ssp126_train_ECs_x1,tf.float64)
ipsl_ssp126_train_ECs_l2 = ipsl_ssp126_train_ECs[:,1].reshape(28854,1)
ipsl_ssp126_train_ECs_x2 = tf.convert_to_tensor(ipsl_ssp126_train_ECs_l2)
ipsl_ssp126_train_ECs_x2=tf.cast(ipsl_ssp126_train_ECs_x2,tf.float64)
ipsl_ssp126_train_ECs_l3 = ipsl_ssp126_train_ECs[:,2].reshape(28854,1)
ipsl_ssp126_train_ECs_x3 = tf.convert_to_tensor(ipsl_ssp126_train_ECs_l3)
ipsl_ssp126_train_ECs_x3=tf.cast(ipsl_ssp126_train_ECs_x3,tf.float64)
ipsl_ssp126_train_ECs_l4 = ipsl_ssp126_train_ECs[:,3].reshape(28854,1)
ipsl_ssp126_train_ECs_x4 = tf.convert_to_tensor(ipsl_ssp126_train_ECs_l4)
ipsl_ssp126_train_ECs_x4=tf.cast(ipsl_ssp126_train_ECs_x4,tf.float64)
ipsl_ssp370_train_ECs_l1 = ipsl_ssp370_train_ECs[:,0].reshape(28854,1)
ipsl_ssp370_train_ECs_x1 = tf.convert_to_tensor(ipsl_ssp370_train_ECs_l1)
ipsl_ssp370_train_ECs_x1=tf.cast(ipsl_ssp370_train_ECs_x1,tf.float64)
ipsl_ssp370_train_ECs_l2 = ipsl_ssp370_train_ECs[:,1].reshape(28854,1)
ipsl_ssp370_train_ECs_x2 = tf.convert_to_tensor(ipsl_ssp370_train_ECs_l2)
ipsl_ssp370_train_ECs_x2=tf.cast(ipsl_ssp370_train_ECs_x2,tf.float64)
ipsl_ssp370_train_ECs_l3 = ipsl_ssp370_train_ECs[:,2].reshape(28854,1)
ipsl_ssp370_train_ECs_x3 = tf.convert_to_tensor(ipsl_ssp370_train_ECs_l3)
ipsl_ssp370_train_ECs_x3=tf.cast(ipsl_ssp370_train_ECs_x3,tf.float64)
ipsl_ssp370_train_ECs_l4 = ipsl_ssp370_train_ECs[:,3].reshape(28854,1)
ipsl_ssp370_train_ECs_x4 = tf.convert_to_tensor(ipsl_ssp370_train_ECs_l4)
ipsl_ssp370_train_ECs_x4=tf.cast(ipsl_ssp370_train_ECs_x4,tf.float64)
ipsl_ssp585_train_ECs_l1 = ipsl_ssp585_train_ECs[:,0].reshape(28854,1)
ipsl_ssp585_train_ECs_x1 = tf.convert_to_tensor(ipsl_ssp585_train_ECs_l1)
ipsl_ssp585_train_ECs_x1=tf.cast(ipsl_ssp585_train_ECs_x1,tf.float64)
ipsl_ssp585_train_ECs_l2 = ipsl_ssp585_train_ECs[:,1].reshape(28854,1)
ipsl_ssp585_train_ECs_x2 = tf.convert_to_tensor(ipsl_ssp585_train_ECs_l2)
ipsl_ssp585_train_ECs_x2=tf.cast(ipsl_ssp585_train_ECs_x2,tf.float64)
ipsl_ssp585_train_ECs_l3 = ipsl_ssp585_train_ECs[:,2].reshape(28854,1)
ipsl_ssp585_train_ECs_x3 = tf.convert_to_tensor(ipsl_ssp585_train_ECs_l3)
ipsl_ssp585_train_ECs_x3=tf.cast(ipsl_ssp585_train_ECs_x3,tf.float64)
ipsl_ssp585_train_ECs_l4 = ipsl_ssp585_train_ECs[:,3].reshape(28854,1)
ipsl_ssp585_train_ECs_x4 = tf.convert_to_tensor(ipsl_ssp585_train_ECs_l4)
ipsl_ssp585_train_ECs_x4=tf.cast(ipsl_ssp585_train_ECs_x4,tf.float64)

In [16]:
ipsl_pred_ECs_ssp126 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_lb_ssp126 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_ub_ssp126 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_ssp370 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_lb_ssp370 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_ub_ssp370 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_ssp585 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_lb_ssp585 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_ub_ssp585 = np.zeros((28854,n_input_ECs))
scaler_x_ssp126 = StandardScaler()
ipsl_train_ssp126 = np.c_[ipsl_ssp126_train_ECs_x1,ipsl_ssp126_train_ECs_x2,ipsl_ssp126_train_ECs_x3,ipsl_ssp126_train_ECs_x4]
ipsl_train_ssp126_sc = scaler_x_ssp126.fit_transform(ipsl_train_ssp126)
scaler_x_ssp370 = StandardScaler()
ipsl_train_ssp370 = np.c_[ipsl_ssp370_train_ECs_x1,ipsl_ssp370_train_ECs_x2,ipsl_ssp370_train_ECs_x3,ipsl_ssp370_train_ECs_x4]
ipsl_train_ssp370_sc = scaler_x_ssp370.fit_transform(ipsl_train_ssp370)
scaler_x_ssp585 = StandardScaler()
ipsl_train_ssp585 = np.c_[ipsl_ssp585_train_ECs_x1,ipsl_ssp585_train_ECs_x2,ipsl_ssp585_train_ECs_x3,ipsl_ssp585_train_ECs_x4]
ipsl_train_ssp585_sc = scaler_x_ssp585.fit_transform(ipsl_train_ssp585)
#输出model的predict_y和上下限，进行EC绘图和面绘图验证

In [17]:
#EOF1

for i in range(n_input_ECs):
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(ipsl_train_sc[:,0].min(), ipsl_train_sc[:,0].max(), n_inducing_points)]
    for j in range(1, n_input_ECs):
            inducing_points = np.c_[inducing_points,
                                    np.linspace(ipsl_train_sc[:, j].min(), ipsl_train_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(ipsl_train_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (ipsl_train_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_ipsl_eof.append(model)
    start_time = time.monotonic()
    opt = gf.optimizers.Scipy()
    opt.minimize(model_ipsl_eof[i].training_loss, model_ipsl_eof[i].trainable_variables)
    gf.utilities.print_summary(model_ipsl_eof[i], "notebook")
    opt_time_eof1 = time.monotonic()
    opt_ipsl_eof1_timeperiod = opt_time_eof1 - start_time
    print("The ",i+1," ECs training for obsdata")
    print('opt_ipsl_eof1_timeperiod:',opt_ipsl_eof1_timeperiod)
    y_pred_mean_ssp126, y_pred_var_ssp126= model_ipsl_eof[i].predict_y(ipsl_train_ssp126_sc, full_cov=False)
    print(y_pred_mean_ssp126,obs_train_sc)
    ipsl_pred_mean_ssp126 = scaler_y.inverse_transform(y_pred_mean_ssp126)
    ipsl_pred_ECs_ssp126[:,i] =np.squeeze(ipsl_pred_mean_ssp126)
    print(ipsl_pred_ECs_ssp126[:,i])
    y__pred_conf_ssp126 = 1.96 * np.sqrt(y_pred_var_ssp126)
    y_pred_lb_ssp126 = y_pred_mean_ssp126 - y__pred_conf_ssp126
    y_pred_ub_ssp126 = y_pred_mean_ssp126 + y__pred_conf_ssp126
    ipsl_pred_ECs_lb_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp126))
    ipsl_pred_ECs_ub_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp126))
    y_pred_mean_ssp370, y_pred_var_ssp370= model_ipsl_eof[i].predict_y(ipsl_train_ssp370_sc, full_cov=False)
    print(y_pred_mean_ssp370,obs_train_sc)
    ipsl_pred_mean_ssp370 = scaler_y.inverse_transform(y_pred_mean_ssp370)
    ipsl_pred_ECs_ssp370[:,i] =np.squeeze(ipsl_pred_mean_ssp370)
    print(ipsl_pred_ECs_ssp370[:,i])
    y__pred_conf_ssp370 = 1.96 * np.sqrt(y_pred_var_ssp370)
    y_pred_lb_ssp370 = y_pred_mean_ssp370 - y__pred_conf_ssp370
    y_pred_ub_ssp370 = y_pred_mean_ssp370 + y__pred_conf_ssp370
    ipsl_pred_ECs_lb_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp370))
    ipsl_pred_ECs_ub_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp370))
    y_pred_mean_ssp585, y_pred_var_ssp585= model_ipsl_eof[i].predict_y(ipsl_train_ssp585_sc, full_cov=False)
    print(y_pred_mean_ssp585,obs_train_sc)
    ipsl_pred_mean_ssp585 = scaler_y.inverse_transform(y_pred_mean_ssp585)
    ipsl_pred_ECs_ssp585[:,i] =np.squeeze(ipsl_pred_mean_ssp585)
    print(ipsl_pred_ECs_ssp585[:,i])
    y__pred_conf_ssp585 = 1.96 * np.sqrt(y_pred_var_ssp585)
    y_pred_lb_ssp585 = y_pred_mean_ssp585 - y__pred_conf_ssp585
    y_pred_ub_ssp585 = y_pred_mean_ssp585 + y__pred_conf_ssp585
    ipsl_pred_ECs_lb_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp585))
    ipsl_pred_ECs_ub_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp585))
    y_pred_mean, y_pred_var = model_ipsl_eof[i].predict_y(ipsl_train_sc, full_cov=False)
    ipsl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(ipsl_pred_mean) / np.mean(obs_train)
    y = (np.std(ipsl_pred_mean) / np.mean(ipsl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    print(B,y)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.87232
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.82096
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.23818
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.13076, -5.89449, -11.44101..."


The  1  ECs training for obsdata
opt_ipsl_eof1_timeperiod: 9958.358999999997
tf.Tensor(
[[0.67710954]
 [0.64309104]
 [0.569497  ]
 ...
 [0.87450122]
 [0.87606873]
 [0.87757893]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-62601.10901334 -67336.53797593 -77580.94522492 ... -35123.87217966
 -34905.67236891 -34695.44961422]
tf.Tensor(
[[0.74807238]
 [0.73897238]
 [0.71156978]
 ...
 [0.75411018]
 [0.75635495]
 [0.75832013]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-52722.96787067 -53989.70286524 -57804.18763066 ... -51882.49642074
 -51570.02079154 -51296.46478012]
tf.Tensor(
[[0.75384968]
 [0.74995319]
 [0.73685034]
 ...
 [0.73954562]
 [0.74059226]
 [0.74164909]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-51918.75797556 -52461.15670295 -54285.09

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.89401
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.71915
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.64019
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.04068, -6.10387, -11.33384..."


The  2  ECs training for obsdata
opt_ipsl_eof1_timeperiod: 14.672000000020489
tf.Tensor(
[[0.04881623]
 [0.1047351 ]
 [0.20850532]
 ...
 [0.04377139]
 [0.0446502 ]
 [0.04554965]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ 463.6082438  1422.30187801 3201.37653587 ...  377.1177112   392.1842838
  407.6048122 ]
tf.Tensor(
[[-0.03479345]
 [-0.01056283]
 [ 0.0378923 ]
 ...
 [-0.04548659]
 [-0.04208255]
 [-0.03863287]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[ -969.8268304   -554.4081229    276.32439281 ... -1153.15381186
 -1094.79378216 -1035.65113248]
tf.Tensor(
[[0.08379984]
 [0.09753716]
 [0.11933516]
 ...
 [0.10218072]
 [0.10728982]
 [0.11204441]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[1063.38006927 1298.89767295 1672.61

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.46843
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.5666
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.78989
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.12127, -5.96236, -11.32813..."


The  3  ECs training for obsdata
opt_ipsl_eof1_timeperiod: 14.09399999998277
tf.Tensor(
[[-0.13302911]
 [-0.15997626]
 [-0.22242806]
 ...
 [ 0.11300612]
 [ 0.11305878]
 [ 0.11311573]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ -459.6742374   -796.63146665 -1577.55194597 ...  2616.84162285
  2617.50004046  2618.21215443]
tf.Tensor(
[[-0.0055809 ]
 [-0.00798328]
 [-0.01911659]
 ...
 [-0.02615281]
 [-0.02368232]
 [-0.02157705]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[1133.98543104 1103.94523643  964.73016136 ...  876.74663253  907.63859553
  933.96369537]
tf.Tensor(
[[0.09619304]
 [0.09411705]
 [0.08607425]
 ...
 [0.15410724]
 [0.15407451]
 [0.15391206]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[2406.60458588 2380.64559792 2280.07552875 ... 3

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,0.53952
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,0.60839
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.76806
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.11801, -5.91728, -11.4055..."


The  4  ECs training for obsdata
opt_ipsl_eof1_timeperiod: 18.53100000001723
tf.Tensor(
[[ 0.03152328]
 [ 0.10575747]
 [ 0.25160669]
 ...
 [-0.18762554]
 [-0.18876676]
 [-0.1897435 ]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[2149.74429633 2782.41830871 4025.44415418 ...  282.0096203   272.28337994
  263.95895255]
tf.Tensor(
[[-0.06521098]
 [-0.03785966]
 [ 0.02969522]
 ...
 [-0.0841004 ]
 [-0.08520267]
 [-0.08601178]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[1325.30942228 1558.41592786 2134.16436454 ... 1164.32101029 1154.92666399
 1148.03093941]
tf.Tensor(
[[0.05089166]
 [0.06413068]
 [0.09623534]
 ...
 [0.10791433]
 [0.10683998]
 [0.10564582]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[2314.81474233 2427.64664585 2701.26

In [18]:
#合并校正后的两个主成分-
ipsl_ssp126_train_ECs_SGPRcor=tf.concat([ipsl_pred_ECs_ssp126[:,0].reshape(28854, 1),ipsl_pred_ECs_ssp126[:,1].reshape(28854, 1),ipsl_pred_ECs_ssp126[:,2].reshape(28854, 1),ipsl_pred_ECs_ssp126[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp126_train_ECs_lb_SGPRcor=tf.concat([ipsl_pred_ECs_lb_ssp126[:,0].reshape(28854, 1),ipsl_pred_ECs_lb_ssp126[:,1].reshape(28854, 1),ipsl_pred_ECs_lb_ssp126[:,2].reshape(28854, 1),ipsl_pred_ECs_lb_ssp126[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp126_train_ECs_ub_SGPRcor=tf.concat([ipsl_pred_ECs_ub_ssp126[:,0].reshape(28854, 1),ipsl_pred_ECs_ub_ssp126[:,1].reshape(28854, 1),ipsl_pred_ECs_ub_ssp126[:,2].reshape(28854, 1),ipsl_pred_ECs_ub_ssp126[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp370_train_ECs_SGPRcor=tf.concat([ipsl_pred_ECs_ssp370[:,0].reshape(28854, 1),ipsl_pred_ECs_ssp370[:,1].reshape(28854, 1),ipsl_pred_ECs_ssp370[:,2].reshape(28854, 1),ipsl_pred_ECs_ssp370[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp370_train_ECs_lb_SGPRcor=tf.concat([ipsl_pred_ECs_lb_ssp370[:,0].reshape(28854, 1),ipsl_pred_ECs_lb_ssp370[:,1].reshape(28854, 1),ipsl_pred_ECs_lb_ssp370[:,2].reshape(28854, 1),ipsl_pred_ECs_lb_ssp370[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp370_train_ECs_ub_SGPRcor=tf.concat([ipsl_pred_ECs_ub_ssp370[:,0].reshape(28854, 1),ipsl_pred_ECs_ub_ssp370[:,1].reshape(28854, 1),ipsl_pred_ECs_ub_ssp370[:,2].reshape(28854, 1),ipsl_pred_ECs_ub_ssp370[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp585_train_ECs_SGPRcor=tf.concat([ipsl_pred_ECs_ssp585[:,0].reshape(28854, 1),ipsl_pred_ECs_ssp585[:,1].reshape(28854, 1),ipsl_pred_ECs_ssp585[:,2].reshape(28854, 1),ipsl_pred_ECs_ssp585[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp585_train_ECs_lb_SGPRcor=tf.concat([ipsl_pred_ECs_lb_ssp585[:,0].reshape(28854, 1),ipsl_pred_ECs_lb_ssp585[:,1].reshape(28854, 1),ipsl_pred_ECs_lb_ssp585[:,2].reshape(28854, 1),ipsl_pred_ECs_lb_ssp585[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp585_train_ECs_ub_SGPRcor=tf.concat([ipsl_pred_ECs_ub_ssp585[:,0].reshape(28854, 1),ipsl_pred_ECs_ub_ssp585[:,1].reshape(28854, 1),ipsl_pred_ECs_ub_ssp585[:,2].reshape(28854, 1),ipsl_pred_ECs_ub_ssp585[:,3].reshape(28854, 1)],axis=1)



In [21]:
#第二次GPFLOW：obs-mean/lb/ub
scaler_z = StandardScaler()
ipsl_pred_ECs_2GP = np.zeros((13149,n_input_ECs))
ipsl_pred_ECs_2GP_lb = np.zeros((13149,n_input_ECs))
ipsl_pred_ECs_2GP_ub = np.zeros((13149,n_input_ECs))
n_inducing_points=700
model_ipsl_eof_2GP = list()
ipsl_pred_ECs_range = np.zeros((13149,n_input_ECs))
ipsl_pred_ECs_2GP_ssp126 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_2GP_lb_ssp126 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_2GP_ub_ssp126 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_range_ssp126 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_2GP_ssp370 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_2GP_lb_ssp370 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_2GP_ub_ssp370 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_range_ssp370 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_2GP_ssp585 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_2GP_lb_ssp585 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_2GP_ub_ssp585 = np.zeros((28854,n_input_ECs))
ipsl_pred_ECs_range_ssp585 = np.zeros((28854,n_input_ECs))
for i in range(n_input_ECs):
    ipsl_pred_ECs_range[:,i] = np.subtract(ipsl_pred_ECs_ub[:,i],ipsl_pred_ECs_lb[:,i])
    ipsl_train_2GP = np.c_[ipsl_pred_ECs[:,i],ipsl_pred_ECs_lb[:,i],ipsl_pred_ECs_ub[:,i],ipsl_pred_ECs_range[:,i] ]
    ipsl_train_2GP_sc = scaler_z.fit_transform(ipsl_train_2GP)
    ipsl_pred_ECs_range_ssp126[:,i] = np.subtract(ipsl_pred_ECs_ub_ssp126[:,i],ipsl_pred_ECs_lb_ssp126[:,i])
    ipsl_train_2GP_ssp126 = np.c_[ipsl_pred_ECs_ssp126[:,i],ipsl_pred_ECs_lb_ssp126[:,i],ipsl_pred_ECs_ub_ssp126[:,i],ipsl_pred_ECs_range_ssp126[:,i] ]
    ipsl_train_2GP_sc_ssp126 = scaler_z.fit_transform(ipsl_train_2GP_ssp126 )
    ipsl_pred_ECs_range_ssp370[:,i] = np.subtract(ipsl_pred_ECs_ub_ssp370[:,i],ipsl_pred_ECs_lb_ssp370[:,i])
    ipsl_train_2GP_ssp370 = np.c_[ipsl_pred_ECs_ssp370[:,i],ipsl_pred_ECs_lb_ssp370[:,i],ipsl_pred_ECs_ub_ssp370[:,i],ipsl_pred_ECs_range_ssp370[:,i] ]
    ipsl_train_2GP_sc_ssp370 = scaler_z.fit_transform(ipsl_train_2GP_ssp370 )
    ipsl_pred_ECs_range_ssp585[:,i] = np.subtract(ipsl_pred_ECs_ub_ssp585[:,i],ipsl_pred_ECs_lb_ssp585[:,i])
    ipsl_train_2GP_ssp585 = np.c_[ipsl_pred_ECs_ssp585[:,i],ipsl_pred_ECs_lb_ssp585[:,i],ipsl_pred_ECs_ub_ssp585[:,i],ipsl_pred_ECs_range_ssp585[:,i] ]
    ipsl_train_2GP_sc_ssp585 = scaler_z.fit_transform(ipsl_train_2GP_ssp585 )
    obs_train = obs_train_all[:,i].reshape(13149,1)
    obs_train_sc = scaler_y.fit_transform(obs_train)
    inducing_points = np.c_[np.linspace(ipsl_train_2GP_sc[:,0].min(), ipsl_train_2GP_sc[:,0].max(), n_inducing_points)]
    for j in range(3):
        inducing_points = np.c_[inducing_points,
                                    np.linspace(ipsl_train_2GP_sc[:, j].min(), ipsl_train_2GP_sc[:, j].max(), n_inducing_points)]
    # Find initial lengthscale
    ini_length = np.mean(abs(ipsl_train_2GP_sc))
    # Create kernel
    k_exp = gf.kernels.SquaredExponential(variance = 1,lengthscales=ini_length)
    model = gf.models.SGPR(
        (ipsl_train_2GP_sc, obs_train_sc),
        kernel=k_exp,
        inducing_variable=inducing_points,
    )
    model_ipsl_eof_2GP.append(model)
    start_time_2GP = time.monotonic()
    opt_2GP = gf.optimizers.Scipy()
    opt_2GP.minimize(model_ipsl_eof_2GP[i].training_loss, model_ipsl_eof_2GP[i].trainable_variables)
    gf.utilities.print_summary(model_ipsl_eof_2GP[i], "notebook")
    opt_time_eof1_2GP = time.monotonic()
    opt_ipsl_eof1_2GP_timeperiod = opt_time_eof1_2GP - start_time_2GP
    print("The ",i+1," ECs training for obsdata")
    print('opt_ipsl_eof1_2GP_timeperiod:',opt_ipsl_eof1_timeperiod)
    y_pred_mean, y_pred_var = model_ipsl_eof_2GP[i].predict_y(ipsl_train_2GP_sc, full_cov=False)
    ipsl_pred_mean = scaler_y.inverse_transform(y_pred_mean)
    B = np.mean(ipsl_pred_mean) / np.mean(obs_train)
    y = (np.std(ipsl_pred_mean) / np.mean(ipsl_pred_mean)) / (np.std(obs_train) / np.mean(obs_train))
    ipsl_pred_ECs_2GP[:,i] =np.squeeze(ipsl_pred_mean)
    print(ipsl_pred_ECs_2GP[:,i])
    y_pred_conf = 1.96 * np.sqrt(y_pred_var)
    y_pred_lb = y_pred_mean - y__pred_conf
    y_pred_ub = y_pred_mean + y__pred_conf
    ipsl_pred_ECs_2GP_lb[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb))
    ipsl_pred_ECs_2GP_ub[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub))
    print(B,y)
    y_pred_mean_2GP_ssp126, y_pred_var_2GP_ssp126= model_ipsl_eof_2GP[i].predict_y(ipsl_train_2GP_sc_ssp126, full_cov=False)
    print(y_pred_mean_ssp126,obs_train_sc)
    ipsl_pred_mean_2GP_ssp126 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp126)
    ipsl_pred_ECs_2GP_ssp126[:,i] =np.squeeze(ipsl_pred_mean_2GP_ssp126)
    print(ipsl_pred_ECs_2GP_ssp126[:,i])
    y__pred_conf_ssp126 = 1.96 * np.sqrt(y_pred_var_ssp126)
    y_pred_lb_ssp126 = y_pred_mean_ssp126 - y__pred_conf_ssp126
    y_pred_ub_ssp126 = y_pred_mean_ssp126 + y__pred_conf_ssp126
    ipsl_pred_ECs_2GP_lb_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp126))
    ipsl_pred_ECs_2GP_ub_ssp126[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp126))
    y_pred_mean_2GP_ssp370, y_pred_var_2GP_ssp370= model_ipsl_eof_2GP[i].predict_y(ipsl_train_2GP_sc_ssp370, full_cov=False)
    print(y_pred_mean_ssp370,obs_train_sc)
    ipsl_pred_mean_2GP_ssp370 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp370)
    ipsl_pred_ECs_2GP_ssp370[:,i] =np.squeeze(ipsl_pred_mean_2GP_ssp370)
    print(ipsl_pred_ECs_2GP_ssp370[:,i])
    y__pred_conf_ssp370 = 1.96 * np.sqrt(y_pred_var_ssp370)
    y_pred_lb_ssp370 = y_pred_mean_ssp370 - y__pred_conf_ssp370
    y_pred_ub_ssp370 = y_pred_mean_ssp370 + y__pred_conf_ssp370
    ipsl_pred_ECs_2GP_lb_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp370))
    ipsl_pred_ECs_2GP_ub_ssp370[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp370))
    y_pred_mean_2GP_ssp585, y_pred_var_2GP_ssp585= model_ipsl_eof_2GP[i].predict_y(ipsl_train_2GP_sc_ssp585, full_cov=False)
    print(y_pred_mean_ssp585,obs_train_sc)
    ipsl_pred_mean_2GP_ssp585 = scaler_y.inverse_transform(y_pred_mean_2GP_ssp585)
    ipsl_pred_ECs_2GP_ssp585[:,i] =np.squeeze(ipsl_pred_mean_2GP_ssp585)
    print(ipsl_pred_ECs_2GP_ssp585[:,i])
    y__pred_conf_ssp585 = 1.96 * np.sqrt(y_pred_var_ssp585)
    y_pred_lb_ssp585 = y_pred_mean_ssp585 - y__pred_conf_ssp585
    y_pred_ub_ssp585 = y_pred_mean_ssp585 + y__pred_conf_ssp585
    ipsl_pred_ECs_2GP_lb_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_lb_ssp585))
    ipsl_pred_ECs_2GP_ub_ssp585[:,i] =np.squeeze(scaler_y.inverse_transform(y_pred_ub_ssp585))

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,145.93536
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,38.29106
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.22585
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.72078, 0.15829, -5.71596..."


The  1  ECs training for obsdata
opt_ipsl_eof1_2GP_timeperiod: 18.53100000001723
[-42615.55552069 -41118.00191436 -39480.60804174 ... -36352.63998426
 -36085.75380009 -35832.56165732]
0.9999982938574387 0.8798263357205106
tf.Tensor(
[[ 0.03152328]
 [ 0.10575747]
 [ 0.25160669]
 ...
 [-0.18762554]
 [-0.18876676]
 [-0.1897435 ]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-64171.30607852 -68740.37251568 -78665.31845128 ... -37921.98208899
 -37715.1661618  -37515.91580053]
tf.Tensor(
[[-0.06521098]
 [-0.03785966]
 [ 0.02969522]
 ...
 [-0.0841004 ]
 [-0.08520267]
 [-0.08601178]], shape=(28854, 1), dtype=float64) [[0.9344392 ]
 [0.93785227]
 [0.94101701]
 ...
 [0.89890814]
 [0.90080121]
 [0.90314889]]
[-55213.64856637 -56535.99202009 -60522.27425074 ... -54333.47158064
 -54007.57842534 -53722.3414539 ]
tf.Tensor(
[[0.05089166]
 [0.06413068]
 [0.09623534]
 ...
 [0.10791433]
 [0.10683998]
 [0.10564582]], shape=(2

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,6.51259
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,13.68566
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.60503
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-4.19477, -4.23335, -4.25392..."


The  2  ECs training for obsdata
opt_ipsl_eof1_2GP_timeperiod: 18.53100000001723
[2727.8993792  2408.33223096 2041.59556497 ...  837.98371057  830.98771772
  826.99567712]
0.9999085843314818 0.6282687608271831
tf.Tensor(
[[ 0.03152328]
 [ 0.10575747]
 [ 0.25160669]
 ...
 [-0.18762554]
 [-0.18876676]
 [-0.1897435 ]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[  22.76640737 1130.49732287 3180.41687069 ...  -80.63296538  -63.23273554
  -45.4221495 ]
tf.Tensor(
[[-0.06521098]
 [-0.03785966]
 [ 0.02969522]
 ...
 [-0.0841004 ]
 [-0.08520267]
 [-0.08601178]], shape=(28854, 1), dtype=float64) [[-0.09716927]
 [-0.09387199]
 [-0.09088116]
 ...
 [-0.04677869]
 [-0.05665051]
 [-0.06543115]]
[-292.83776233  163.30133943 1075.23502063 ... -493.32745212 -429.26881225
 -364.35508236]
tf.Tensor(
[[0.05089166]
 [0.06413068]
 [0.09623534]
 ...
 [0.10791433]
 [0.10683998]
 [0.10564582]], shape=(28854, 1), dtype=float64)

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,8.06187
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,7.54636
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.73086
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-7.32953, -7.30265, -7.27027..."


The  3  ECs training for obsdata
opt_ipsl_eof1_2GP_timeperiod: 18.53100000001723
[2818.40528937 2878.76379584 2922.40636245 ... 2782.90213043 2781.1264983
 2780.14468632]
1.0001194978970809 0.5181584261437613
tf.Tensor(
[[ 0.03152328]
 [ 0.10575747]
 [ 0.25160669]
 ...
 [-0.18762554]
 [-0.18876676]
 [-0.1897435 ]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[ -538.24532745  -948.79611228 -1905.77951244 ...  3246.64831373
  3247.47273907  3248.36667902]
tf.Tensor(
[[-0.06521098]
 [-0.03785966]
 [ 0.02969522]
 ...
 [-0.0841004 ]
 [-0.08520267]
 [-0.08601178]], shape=(28854, 1), dtype=float64) [[0.01163053]
 [0.01864558]
 [0.02472877]
 ...
 [0.07700931]
 [0.0561183 ]
 [0.0380782 ]]
[713.91256794 673.65795254 487.11467339 ... 368.54363088 409.98652097
 445.30462485]
tf.Tensor(
[[0.05089166]
 [0.06413068]
 [0.09623534]
 ...
 [0.10791433]
 [0.10683998]
 [0.10564582]], shape=(28854, 1), dtype=float64) [[0.01163053

name,class,transform,prior,trainable,shape,dtype,value
SGPR.kernel.variance,Parameter,Softplus,,True,(),float64,33.03433
SGPR.kernel.lengthscales,Parameter,Softplus,,True,(),float64,29.25466
SGPR.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,0.72035
SGPR.inducing_variable.Z,Parameter,Identity,,True,"(700, 4)",float64,"[[-5.58614, -4.80079, -7.03631..."


The  4  ECs training for obsdata
opt_ipsl_eof1_2GP_timeperiod: 18.53100000001723
[1705.17947265 1440.70062042 1149.84257357 ...  191.72024195  170.02359763
  151.76456158]
1.000050659240034 0.5285126489623638
tf.Tensor(
[[ 0.03152328]
 [ 0.10575747]
 [ 0.25160669]
 ...
 [-0.18762554]
 [-0.18876676]
 [-0.1897435 ]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[2041.95930268 2797.19016772 4263.73453881 ... -222.2577891  -234.15892038
 -244.34381585]
tf.Tensor(
[[-0.06521098]
 [-0.03785966]
 [ 0.02969522]
 ...
 [-0.0841004 ]
 [-0.08520267]
 [-0.08601178]], shape=(28854, 1), dtype=float64) [[-0.35770041]
 [-0.35268431]
 [-0.3476021 ]
 ...
 [-0.27010257]
 [-0.28864854]
 [-0.30485149]]
[ 413.55825951  710.87344562 1442.07800676 ...  207.92828689  195.87969637
  187.03145643]
tf.Tensor(
[[0.05089166]
 [0.06413068]
 [0.09623534]
 ...
 [0.10791433]
 [0.10683998]
 [0.10564582]], shape=(28854, 1), dtype=float64) 

In [22]:
#合并校正后的两个主成分-
ipsl_ssp126_train_ECs_2GP_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_ssp126[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp126[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp126[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp126[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp126_train_ECs_2GP_lb_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_lb_ssp126[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp126[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp126[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp126[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp126_train_ECs_2GP_ub_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_ub_ssp126[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp126[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp126[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp126[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp370_train_ECs_2GP_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_ssp370[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp370[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp370[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp370[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp370_train_ECs_2GP_lb_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_lb_ssp370[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp370[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp370[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp370[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp370_train_ECs_2GP_ub_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_ub_ssp370[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp370[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp370[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp370[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp585_train_ECs_2GP_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_ssp585[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp585[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp585[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_ssp585[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp585_train_ECs_2GP_lb_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_lb_ssp585[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp585[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp585[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_lb_ssp585[:,3].reshape(28854, 1)],axis=1)
ipsl_ssp585_train_ECs_2GP_ub_SGPRcor=tf.concat([ipsl_pred_ECs_2GP_ub_ssp585[:,0].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp585[:,1].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp585[:,2].reshape(28854, 1),ipsl_pred_ECs_2GP_ub_ssp585[:,3].reshape(28854, 1)],axis=1)

# 写入Excel文件
df_mean_merge_2GP_ssp126 = pd.DataFrame(ipsl_ssp126_train_ECs_2GP_SGPRcor,columns = ['ipsl_EC1_2GP','ipsl_EC2_2GP','ipsl_EC3_2GP','ipsl_EC4_2GP'])
df_lb_merge_2GP_ssp126 = pd.DataFrame(ipsl_ssp126_train_ECs_2GP_lb_SGPRcor,columns = ['ipsl_EC1_2GP_lb','ipsl_EC2_2GP_lb','ipsl_EC3_2GP_lb','ipsl_EC4_2GP_lb'])
df_ub_merge_2GP_ssp126 = pd.DataFrame(ipsl_ssp126_train_ECs_2GP_ub_SGPRcor,columns = ['ipsl_EC1_2GP_ub','ipsl_EC2_2GP_ub','ipsl_EC3_2GP_ub','ipsl_EC4_2GP_ub'])
df_mean_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_mean_ssp126_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_lb_ssp126_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp126.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_ub_ssp126_2GP.xlsx',index=False)
df_mean_merge_2GP_ssp370 = pd.DataFrame(ipsl_ssp370_train_ECs_2GP_SGPRcor,columns = ['ipsl_EC1_2GP','ipsl_EC2_2GP','ipsl_EC3_2GP','ipsl_EC4_2GP'])
df_lb_merge_2GP_ssp370 = pd.DataFrame(ipsl_ssp370_train_ECs_2GP_lb_SGPRcor,columns = ['ipsl_EC1_2GP_lb','ipsl_EC2_2GP_lb','ipsl_EC3_2GP_lb','ipsl_EC4_2GP_lb'])
df_ub_merge_2GP_ssp370 = pd.DataFrame(ipsl_ssp370_train_ECs_2GP_ub_SGPRcor,columns = ['ipsl_EC1_2GP_ub','ipsl_EC2_2GP_ub','ipsl_EC3_2GP_ub','ipsl_EC4_2GP_ub'])
df_mean_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_mean_ssp370_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_lb_ssp370_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp370.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_ub_ssp370_2GP.xlsx',index=False)
df_mean_merge_2GP_ssp585 = pd.DataFrame(ipsl_ssp585_train_ECs_2GP_SGPRcor,columns = ['ipsl_EC1_2GP','ipsl_EC2_2GP','ipsl_EC3_2GP','ipsl_EC4_2GP'])
df_lb_merge_2GP_ssp585 = pd.DataFrame(ipsl_ssp585_train_ECs_2GP_lb_SGPRcor,columns = ['ipsl_EC1_2GP_lb','ipsl_EC2_2GP_lb','ipsl_EC3_2GP_lb','ipsl_EC4_2GP_lb'])
df_ub_merge_2GP_ssp585 = pd.DataFrame(ipsl_ssp585_train_ECs_2GP_ub_SGPRcor,columns = ['ipsl_EC1_2GP_ub','ipsl_EC2_2GP_ub','ipsl_EC3_2GP_ub','ipsl_EC4_2GP_ub'])
df_mean_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_mean_ssp585_2GP.xlsx',index=False)
df_lb_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_lb_ssp585_2GP.xlsx',index=False)
df_ub_merge_2GP_ssp585.to_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_ub_ssp585_2GP.xlsx',index=False)

In [2]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_mean_ssp126_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp126_train_SGPRcor_EC1_2GP = data_frame['ipsl_EC1_2GP']
ipsl_ssp126_train_SGPRcor_EC1_2GP_array = np.array(ipsl_ssp126_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC1_2GP_array)
ipsl_ssp126_train_SGPRcor_EC1_2GP_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC1_2GP_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC2_2GP = data_frame['ipsl_EC2_2GP']
ipsl_ssp126_train_SGPRcor_EC2_2GP_array = np.array(ipsl_ssp126_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC2_2GP_array)
ipsl_ssp126_train_SGPRcor_EC2_2GP_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC2_2GP_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC3_2GP = data_frame['ipsl_EC3_2GP']
ipsl_ssp126_train_SGPRcor_EC3_2GP_array = np.array(ipsl_ssp126_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC3_2GP_array)
ipsl_ssp126_train_SGPRcor_EC3_2GP_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC3_2GP_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC4_2GP = data_frame['ipsl_EC4_2GP']
ipsl_ssp126_train_SGPRcor_EC4_2GP_array = np.array(ipsl_ssp126_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC4_2GP_array)
ipsl_ssp126_train_SGPRcor_EC4_2GP_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC4_2GP_tf,tf.float64)
ipsl_ssp126_train_ECs_2GP_SGPRcor=tf.concat([ipsl_ssp126_train_SGPRcor_EC1_2GP_tf,ipsl_ssp126_train_SGPRcor_EC2_2GP_tf,ipsl_ssp126_train_SGPRcor_EC3_2GP_tf,ipsl_ssp126_train_SGPRcor_EC4_2GP_tf],axis=1)
print(ipsl_ssp126_train_ECs_2GP_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_mean_ssp370_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp370_train_SGPRcor_EC1_2GP = data_frame['ipsl_EC1_2GP']
ipsl_ssp370_train_SGPRcor_EC1_2GP_array = np.array(ipsl_ssp370_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC1_2GP_array)
ipsl_ssp370_train_SGPRcor_EC1_2GP_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC1_2GP_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC2_2GP = data_frame['ipsl_EC2_2GP']
ipsl_ssp370_train_SGPRcor_EC2_2GP_array = np.array(ipsl_ssp370_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC2_2GP_array)
ipsl_ssp370_train_SGPRcor_EC2_2GP_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC2_2GP_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC3_2GP = data_frame['ipsl_EC3_2GP']
ipsl_ssp370_train_SGPRcor_EC3_2GP_array = np.array(ipsl_ssp370_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC3_2GP_array)
ipsl_ssp370_train_SGPRcor_EC3_2GP_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC3_2GP_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC4_2GP = data_frame['ipsl_EC4_2GP']
ipsl_ssp370_train_SGPRcor_EC4_2GP_array = np.array(ipsl_ssp370_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC4_2GP_array)
ipsl_ssp370_train_SGPRcor_EC4_2GP_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC4_2GP_tf,tf.float64)
ipsl_ssp370_train_ECs_2GP_SGPRcor=tf.concat([ipsl_ssp370_train_SGPRcor_EC1_2GP_tf,ipsl_ssp370_train_SGPRcor_EC2_2GP_tf,ipsl_ssp370_train_SGPRcor_EC3_2GP_tf,ipsl_ssp370_train_SGPRcor_EC4_2GP_tf],axis=1)
print(ipsl_ssp370_train_ECs_2GP_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_mean_ssp585_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp585_train_SGPRcor_EC1_2GP = data_frame['ipsl_EC1_2GP']
ipsl_ssp585_train_SGPRcor_EC1_2GP_array = np.array(ipsl_ssp585_train_SGPRcor_EC1_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC1_2GP_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC1_2GP_array)
ipsl_ssp585_train_SGPRcor_EC1_2GP_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC1_2GP_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC2_2GP = data_frame['ipsl_EC2_2GP']
ipsl_ssp585_train_SGPRcor_EC2_2GP_array = np.array(ipsl_ssp585_train_SGPRcor_EC2_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC2_2GP_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC2_2GP_array)
ipsl_ssp585_train_SGPRcor_EC2_2GP_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC2_2GP_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC3_2GP = data_frame['ipsl_EC3_2GP']
ipsl_ssp585_train_SGPRcor_EC3_2GP_array = np.array(ipsl_ssp585_train_SGPRcor_EC3_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC3_2GP_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC3_2GP_array)
ipsl_ssp585_train_SGPRcor_EC3_2GP_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC3_2GP_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC4_2GP = data_frame['ipsl_EC4_2GP']
ipsl_ssp585_train_SGPRcor_EC4_2GP_array = np.array(ipsl_ssp585_train_SGPRcor_EC4_2GP.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC4_2GP_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC4_2GP_array)
ipsl_ssp585_train_SGPRcor_EC4_2GP_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC4_2GP_tf,tf.float64)
ipsl_ssp585_train_ECs_2GP_SGPRcor=tf.concat([ipsl_ssp585_train_SGPRcor_EC1_2GP_tf,ipsl_ssp585_train_SGPRcor_EC2_2GP_tf,ipsl_ssp585_train_SGPRcor_EC3_2GP_tf,ipsl_ssp585_train_SGPRcor_EC4_2GP_tf],axis=1)
print(ipsl_ssp585_train_ECs_2GP_SGPRcor)

tf.Tensor(
[[-6.41713061e+04  2.27664074e+01 -5.38245327e+02  2.04195930e+03]
 [-6.87403725e+04  1.13049732e+03 -9.48796112e+02  2.79719017e+03]
 [-7.86653185e+04  3.18041687e+03 -1.90577951e+03  4.26373454e+03]
 ...
 [-3.79219821e+04 -8.06329654e+01  3.24664831e+03 -2.22257789e+02]
 [-3.77151662e+04 -6.32327355e+01  3.24747274e+03 -2.34158920e+02]
 [-3.75159158e+04 -4.54221495e+01  3.24836668e+03 -2.44343816e+02]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-55213.64856637   -292.83776233    713.91256794    413.55825951]
 [-56535.99202009    163.30133943    673.65795254    710.87344562]
 [-60522.27425074   1075.23502063    487.11467339   1442.07800676]
 ...
 [-54333.47158064   -493.32745212    368.54363088    207.92828689]
 [-54007.57842534   -429.26881225    409.98652097    195.87969637]
 [-53722.3414539    -364.35508236    445.30462485    187.03145643]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-55497.69759064    882.95311933   1519.40138613    924.9931207 ]
 [-56069.5868758

In [3]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_lb_ssp126_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp126_train_SGPRcor_EC1_2GP_lb = data_frame['ipsl_EC1_2GP_lb']
ipsl_ssp126_train_SGPRcor_EC1_2GP_lb_array = np.array(ipsl_ssp126_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC1_2GP_lb_array)
ipsl_ssp126_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC2_2GP_lb = data_frame['ipsl_EC2_2GP_lb']
ipsl_ssp126_train_SGPRcor_EC2_2GP_lb_array = np.array(ipsl_ssp126_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC2_2GP_lb_array)
ipsl_ssp126_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC3_2GP_lb = data_frame['ipsl_EC3_2GP_lb']
ipsl_ssp126_train_SGPRcor_EC3_2GP_lb_array = np.array(ipsl_ssp126_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC3_2GP_lb_array)
ipsl_ssp126_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC4_2GP_lb = data_frame['ipsl_EC4_2GP_lb']
ipsl_ssp126_train_SGPRcor_EC4_2GP_lb_array = np.array(ipsl_ssp126_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC4_2GP_lb_array)
ipsl_ssp126_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
ipsl_ssp126_train_ECs_2GP_lb_SGPRcor=tf.concat([ipsl_ssp126_train_SGPRcor_EC1_2GP_lb_tf,ipsl_ssp126_train_SGPRcor_EC2_2GP_lb_tf,ipsl_ssp126_train_SGPRcor_EC3_2GP_lb_tf,ipsl_ssp126_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(ipsl_ssp126_train_ECs_2GP_lb_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_lb_ssp370_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp370_train_SGPRcor_EC1_2GP_lb = data_frame['ipsl_EC1_2GP_lb']
ipsl_ssp370_train_SGPRcor_EC1_2GP_lb_array = np.array(ipsl_ssp370_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC1_2GP_lb_array)
ipsl_ssp370_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC2_2GP_lb = data_frame['ipsl_EC2_2GP_lb']
ipsl_ssp370_train_SGPRcor_EC2_2GP_lb_array = np.array(ipsl_ssp370_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC2_2GP_lb_array)
ipsl_ssp370_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC3_2GP_lb = data_frame['ipsl_EC3_2GP_lb']
ipsl_ssp370_train_SGPRcor_EC3_2GP_lb_array = np.array(ipsl_ssp370_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC3_2GP_lb_array)
ipsl_ssp370_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC4_2GP_lb = data_frame['ipsl_EC4_2GP_lb']
ipsl_ssp370_train_SGPRcor_EC4_2GP_lb_array = np.array(ipsl_ssp370_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC4_2GP_lb_array)
ipsl_ssp370_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
ipsl_ssp370_train_ECs_2GP_lb_SGPRcor=tf.concat([ipsl_ssp370_train_SGPRcor_EC1_2GP_lb_tf,ipsl_ssp370_train_SGPRcor_EC2_2GP_lb_tf,ipsl_ssp370_train_SGPRcor_EC3_2GP_lb_tf,ipsl_ssp370_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(ipsl_ssp370_train_ECs_2GP_lb_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_lb_ssp585_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp585_train_SGPRcor_EC1_2GP_lb = data_frame['ipsl_EC1_2GP_lb']
ipsl_ssp585_train_SGPRcor_EC1_2GP_lb_array = np.array(ipsl_ssp585_train_SGPRcor_EC1_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC1_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC1_2GP_lb_array)
ipsl_ssp585_train_SGPRcor_EC1_2GP_lb_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC1_2GP_lb_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC2_2GP_lb = data_frame['ipsl_EC2_2GP_lb']
ipsl_ssp585_train_SGPRcor_EC2_2GP_lb_array = np.array(ipsl_ssp585_train_SGPRcor_EC2_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC2_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC2_2GP_lb_array)
ipsl_ssp585_train_SGPRcor_EC2_2GP_lb_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC2_2GP_lb_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC3_2GP_lb = data_frame['ipsl_EC3_2GP_lb']
ipsl_ssp585_train_SGPRcor_EC3_2GP_lb_array = np.array(ipsl_ssp585_train_SGPRcor_EC3_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC3_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC3_2GP_lb_array)
ipsl_ssp585_train_SGPRcor_EC3_2GP_lb_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC3_2GP_lb_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC4_2GP_lb = data_frame['ipsl_EC4_2GP_lb']
ipsl_ssp585_train_SGPRcor_EC4_2GP_lb_array = np.array(ipsl_ssp585_train_SGPRcor_EC4_2GP_lb.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC4_2GP_lb_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC4_2GP_lb_array)
ipsl_ssp585_train_SGPRcor_EC4_2GP_lb_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC4_2GP_lb_tf,tf.float64)
ipsl_ssp585_train_ECs_2GP_lb_SGPRcor=tf.concat([ipsl_ssp585_train_SGPRcor_EC1_2GP_lb_tf,ipsl_ssp585_train_SGPRcor_EC2_2GP_lb_tf,ipsl_ssp585_train_SGPRcor_EC3_2GP_lb_tf,ipsl_ssp585_train_SGPRcor_EC4_2GP_lb_tf],axis=1)
print(ipsl_ssp585_train_ECs_2GP_lb_SGPRcor)

tf.Tensor(
[[-391936.70552102  -29326.37919404  -19913.3612871   -12511.84570225]
 [-381653.61240557  -28059.89139824  -18989.6390039   -11882.25891072]
 [-361474.18198362  -25574.54948132  -17176.93641201  -10646.76461815]
 ...
 [-422177.83689683  -33050.94177346  -22629.89867594  -14363.37195619]
 [-422336.41799534  -33070.47296156  -22644.14389309  -14373.08115204]
 [-422472.32283649  -33087.21129319  -22656.35211962  -14381.40198429]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-405257.68146256  -30967.01915214  -21109.97422963  -13327.42817426]
 [-401465.62788189  -30499.9817027   -20769.33699604  -13095.25806548]
 [-392126.35196855  -29349.73645713  -19930.39708047  -12523.45688743]
 ...
 [-407890.23000099  -31291.2494745   -21346.45402411  -13488.60708893]
 [-408040.95060234  -31309.81254692  -21359.99313889  -13497.83502247]
 [-408151.3133731   -31323.4050624   -21369.90694104  -13504.5920305 ]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[-389261.01766622  -28996.83574097

In [4]:
# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_ub_ssp126_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp126_train_SGPRcor_EC1_2GP_ub = data_frame['ipsl_EC1_2GP_ub']
ipsl_ssp126_train_SGPRcor_EC1_2GP_ub_array = np.array(ipsl_ssp126_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC1_2GP_ub_array)
ipsl_ssp126_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC2_2GP_ub = data_frame['ipsl_EC2_2GP_ub']
ipsl_ssp126_train_SGPRcor_EC2_2GP_ub_array = np.array(ipsl_ssp126_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC2_2GP_ub_array)
ipsl_ssp126_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC3_2GP_ub = data_frame['ipsl_EC3_2GP_ub']
ipsl_ssp126_train_SGPRcor_EC3_2GP_ub_array = np.array(ipsl_ssp126_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC3_2GP_ub_array)
ipsl_ssp126_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
ipsl_ssp126_train_SGPRcor_EC4_2GP_ub = data_frame['ipsl_EC4_2GP_ub']
ipsl_ssp126_train_SGPRcor_EC4_2GP_ub_array = np.array(ipsl_ssp126_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp126_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp126_train_SGPRcor_EC4_2GP_ub_array)
ipsl_ssp126_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(ipsl_ssp126_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
ipsl_ssp126_train_ECs_2GP_ub_SGPRcor=tf.concat([ipsl_ssp126_train_SGPRcor_EC1_2GP_ub_tf,ipsl_ssp126_train_SGPRcor_EC2_2GP_ub_tf,ipsl_ssp126_train_SGPRcor_EC3_2GP_ub_tf,ipsl_ssp126_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(ipsl_ssp126_train_ECs_2GP_ub_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_ub_ssp370_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp370_train_SGPRcor_EC1_2GP_ub = data_frame['ipsl_EC1_2GP_ub']
ipsl_ssp370_train_SGPRcor_EC1_2GP_ub_array = np.array(ipsl_ssp370_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC1_2GP_ub_array)
ipsl_ssp370_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC2_2GP_ub = data_frame['ipsl_EC2_2GP_ub']
ipsl_ssp370_train_SGPRcor_EC2_2GP_ub_array = np.array(ipsl_ssp370_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC2_2GP_ub_array)
ipsl_ssp370_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC3_2GP_ub = data_frame['ipsl_EC3_2GP_ub']
ipsl_ssp370_train_SGPRcor_EC3_2GP_ub_array = np.array(ipsl_ssp370_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC3_2GP_ub_array)
ipsl_ssp370_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
ipsl_ssp370_train_SGPRcor_EC4_2GP_ub = data_frame['ipsl_EC4_2GP_ub']
ipsl_ssp370_train_SGPRcor_EC4_2GP_ub_array = np.array(ipsl_ssp370_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp370_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp370_train_SGPRcor_EC4_2GP_ub_array)
ipsl_ssp370_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(ipsl_ssp370_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
ipsl_ssp370_train_ECs_2GP_ub_SGPRcor=tf.concat([ipsl_ssp370_train_SGPRcor_EC1_2GP_ub_tf,ipsl_ssp370_train_SGPRcor_EC2_2GP_ub_tf,ipsl_ssp370_train_SGPRcor_EC3_2GP_ub_tf,ipsl_ssp370_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(ipsl_ssp370_train_ECs_2GP_ub_SGPRcor)

# 读取 Excel 文件
data_frame = pd.read_excel('E:\ZJ组科研\数据源\第二篇文章\EOF+gpflow\ipsl_ECs_pred_ub_ssp585_2GP.xlsx')
# 获取指定列的数据
ipsl_ssp585_train_SGPRcor_EC1_2GP_ub = data_frame['ipsl_EC1_2GP_ub']
ipsl_ssp585_train_SGPRcor_EC1_2GP_ub_array = np.array(ipsl_ssp585_train_SGPRcor_EC1_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC1_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC1_2GP_ub_array)
ipsl_ssp585_train_SGPRcor_EC1_2GP_ub_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC1_2GP_ub_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC2_2GP_ub = data_frame['ipsl_EC2_2GP_ub']
ipsl_ssp585_train_SGPRcor_EC2_2GP_ub_array = np.array(ipsl_ssp585_train_SGPRcor_EC2_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC2_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC2_2GP_ub_array)
ipsl_ssp585_train_SGPRcor_EC2_2GP_ub_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC2_2GP_ub_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC3_2GP_ub = data_frame['ipsl_EC3_2GP_ub']
ipsl_ssp585_train_SGPRcor_EC3_2GP_ub_array = np.array(ipsl_ssp585_train_SGPRcor_EC3_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC3_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC3_2GP_ub_array)
ipsl_ssp585_train_SGPRcor_EC3_2GP_ub_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC3_2GP_ub_tf,tf.float64)
ipsl_ssp585_train_SGPRcor_EC4_2GP_ub = data_frame['ipsl_EC4_2GP_ub']
ipsl_ssp585_train_SGPRcor_EC4_2GP_ub_array = np.array(ipsl_ssp585_train_SGPRcor_EC4_2GP_ub.values.tolist()).reshape(28854,1)
ipsl_ssp585_train_SGPRcor_EC4_2GP_ub_tf = tf.convert_to_tensor(ipsl_ssp585_train_SGPRcor_EC4_2GP_ub_array)
ipsl_ssp585_train_SGPRcor_EC4_2GP_ub_tf=tf.cast(ipsl_ssp585_train_SGPRcor_EC4_2GP_ub_tf,tf.float64)
ipsl_ssp585_train_ECs_2GP_ub_SGPRcor=tf.concat([ipsl_ssp585_train_SGPRcor_EC1_2GP_ub_tf,ipsl_ssp585_train_SGPRcor_EC2_2GP_ub_tf,ipsl_ssp585_train_SGPRcor_EC3_2GP_ub_tf,ipsl_ssp585_train_SGPRcor_EC4_2GP_ub_tf],axis=1)
print(ipsl_ssp585_train_ECs_2GP_ub_SGPRcor)

tf.Tensor(
[[ 87001.21759115  29660.64229406  23109.26107773  16811.3342949 ]
 [ 97385.15837348  30939.55070473  24042.04242867  17447.09552814]
 [117810.61574766  33455.1938286   25876.84543097  18697.65292651]
 ...
 [ 56230.61921944  25870.86942421  20345.16207909  14927.39119679]
 [ 56071.48134205  25851.26966203  20330.86684693  14917.64791191]
 [ 55935.4590619   25834.51686636  20318.64807094  14909.3198894 ]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[73391.06762077 27984.38704256 21886.67185636 15978.04701881]
 [77213.70969441 28455.19183637 22230.05683055 16212.0899212 ]
 [86681.92741078 29621.31783087 23080.57948834 16791.78561651]
 ...
 [70764.74207459 27660.92315726 21650.75106845 15817.24910951]
 [70608.58540654 27641.69056721 21636.72363601 15807.68835045]
 [70493.69129787 27627.53996245 21626.40278729 15800.65390931]], shape=(28854, 4), dtype=float64)
tf.Tensor(
[[89717.74821891 29995.21603061 23353.28491819 16977.65504906]
 [91592.55410137 30226.12114282 23521.697277

In [5]:
#predict
#生成新数据
def ReconstructDataFromEOFandECs(EOFs, ECs):
    # Check if EOF and EC has equally many modes
    if EOFs.shape[0] != ECs.shape[1]:
        return print('EOF and ECs does not have equal number of modes')
    # Project ECs on EOFs
    #recon = np.dot(ECs, EOFs)
    recon = np.zeros((28854,89,189))
    for i in range (len(ECs)):
        recon[i,:,:] = ECs[i,0]*EOFs[0,:,:] + ECs[i,1]*EOFs[1,:,:] + ECs[i,2]*EOFs[2,:,:] + ECs[i,3]*EOFs[3,:,:]
    # Remove weight
    #recon = recon / Weights
    #recon = np.add(recon, Mean)
    # Return reconstructed values
    return recon


In [9]:
recondata_SGPR_ipsl_mix4_2GP_ssp126 = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp126_train_ECs_2GP_SGPRcor)
recondata_SGPR_ipsl_mix4_2GP_ssp370 = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp370_train_ECs_2GP_SGPRcor)
recondata_SGPR_ipsl_mix4_2GP_ssp585 = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp585_train_ECs_2GP_SGPRcor)
recondata_SGPR_ipsl_mix4_2GP_ssp126_lb = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp126_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_ipsl_mix4_2GP_ssp370_lb = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp370_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_ipsl_mix4_2GP_ssp585_lb = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp585_train_ECs_2GP_lb_SGPRcor)
recondata_SGPR_ipsl_mix4_2GP_ssp126_ub = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp126_train_ECs_2GP_ub_SGPRcor)
recondata_SGPR_ipsl_mix4_2GP_ssp370_ub = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp370_train_ECs_2GP_ub_SGPRcor)
recondata_SGPR_ipsl_mix4_2GP_ssp585_ub = ReconstructDataFromEOFandECs(obsdata_eofs,ipsl_ssp585_train_ECs_2GP_ub_SGPRcor)

In [10]:
#将新数据写入文件
def writeNetCDF(myvar,filename):
    lon = np.linspace(82,97.75,189)
    #lat = np.linspace(23.833333333333,31.25,89)
    lat = np.linspace(31.25,23.833333333333,89)
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2100, 12, 31)
    delta = timedelta(days=1)
    dates = np.arange(start_date, end_date + delta, delta).astype(datetime)
    ncout = Dataset(filename,'w','NETCDF4') # using netCDF4 for output format 
    ncout.createDimension('lon',len(lon))
    ncout.createDimension('lat',len(lat))
    ncout.createDimension('time',None)
    lonvar = ncout.createVariable('lon','float32',('lon'))
    lonvar[:] = lon
    latvar = ncout.createVariable('lat','float32',('lat'))
    latvar[:] = lat
    timevar = ncout.createVariable('time','float64',('time',))
    timevar.units = 'days since 2022-01-01 00:00:00'
    timevar.calendar = 'standard'
    timevar[:] = date2num(dates, units=timevar.units, calendar=timevar.calendar)
    var = ncout.createVariable('discharge','float32',('time','lat','lon'))
    var.setncattr('units','m3 s-1')
    var[:] = myvar[:]
    ncout.close()

In [11]:
out_dir='E:/data/era5_runoff/disdata/' 
out_file_nc1 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp126.nc' 
out_file_nc2 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp370.nc' 
out_file_nc3 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp585.nc'
out_file_nc4 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp126_lb.nc' 
out_file_nc5 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp370_lb.nc' 
out_file_nc6 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp585_lb.nc' 
out_file_nc7 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp126_ub.nc' 
out_file_nc8 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp370_ub.nc' 
out_file_nc9 = out_dir+'recondata_SGPR_ipsl_mix4_2GP_ssp585_ub.nc' 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp126,out_file_nc1) 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp370,out_file_nc2) 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp585,out_file_nc3) 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp126_lb,out_file_nc4) 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp370_lb,out_file_nc5) 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp585_lb,out_file_nc6) 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp126_ub,out_file_nc7) 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp370_ub,out_file_nc8) 
writeNetCDF(recondata_SGPR_ipsl_mix4_2GP_ssp585_ub,out_file_nc9) 